In [2]:
import czifile as zis
#from apeer_ometiff_library import io, processing, omexmlClass
import os
import tifffile
import ipywidgets as widgets
from matplotlib import pyplot as plt, cm
from mpl_toolkits.axes_grid1 import make_axes_locatable
import xmltodict
import numpy as np
from collections import Counter
from lxml import etree as ET
import time
import re
from aicsimageio import AICSImage, imread, imread_dask
import dask.array as da
import napari
import pandas as pd

In [24]:
filename = r'E24W-1_C5_09.czi'

# get CZI object and read array
czi = zis.CziFile(filename)

# parse the XML into a dictionary
metadatadict_czi = czi.metadata(raw=False)

print(czi.axes)
print(czi.shape)

# add axes and shape information using aicsimageio package
czi_aics = AICSImage(filename)

print(czi_aics.dims)
print(czi_aics.shape)
print(czi_aics.dims.X)
print(czi_aics.dims.Y)
print(czi_aics.dims.S)
print(czi_aics.dims.C)
print(czi_aics.dims.T)

# determine pixel type for CZI array
print(czi.dtype)

# check if the CZI image is an RGB image depending on the last dimension entry of axes
print(czi.axes[-1])

# check CZI image pixel type
print(metadatadict_czi['ImageDocument']['Metadata']['Information']['Image']['PixelType'])

# checj CZI image size
print(int(metadatadict_czi['ImageDocument']['Metadata']['Information']['Image']['SizeX']))
print(int(metadatadict_czi['ImageDocument']['Metadata']['Information']['Image']['SizeY']))
print(int(metadatadict_czi['ImageDocument']['Metadata']['Information']['Image']['SizeC']))
print(metadatadict_czi['ImageDocument']['Metadata']['DisplaySetting']['Channels']
      ['Channel']['ShortName'])

SCYX0
(30, 1, 62429, 129731, 3)
<Dimensions [T: 1, C: 1, Z: 1, Y: 6864, X: 20545, S: 3]>
(1, 1, 1, 6864, 20545, 3)
20545
6864
3
1
1
uint8
0
Bgr24
129731
62429
1
Brigh


In [ ]:
def get_metadata_czi(filename, dim2none=False):
    """
    Returns a dictionary with CZI metadata.

    Information CZI Dimension Characters:
    - '0': 'Sample',  # e.g. RGBA
    - 'X': 'Width',
    - 'Y': 'Height',
    - 'C': 'Channel',
    - 'Z': 'Slice',  # depth
    - 'T': 'Time',
    - 'R': 'Rotation',
    - 'S': 'Scene',  # contiguous regions of interest in a mosaic image
    - 'I': 'Illumination',  # direction
    - 'B': 'Block',  # acquisition
    - 'M': 'Mosaic',  # index of tile for compositing a scene
    - 'H': 'Phase',  # e.g. Airy detector fibers
    - 'V': 'View',  # e.g. for SPIM

    :param filename: filename of the CZI image
    :type filename: str
    :param dim2none: option to set non-existing dimension to None, defaults to False
    :type dim2none: bool, optional
    :return: metadata - dictionary with the relevant CZI metainformation
    :rtype: dict
    """

    # get CZI object and read array
    czi = zis.CziFile(filename)

    # parse the XML into a dictionary
    metadatadict_czi = czi.metadata(raw=False)

    # parse the XML into a dictionary
    # mdczi = czi.metadata()
    # metadatadict_czi = xmltodict.parse(czi.metadata())
    metadata = create_metadata_dict()

    # get directory and filename etc.
    metadata['Directory'] = os.path.dirname(filename)
    metadata['Filename'] = os.path.basename(filename)
    metadata['Extension'] = 'czi'
    metadata['ImageType'] = 'czi'

    # add axes and shape information using czifile package
    metadata['Axes'] = czi.axes
    metadata['Shape'] = czi.shape

    # add axes and shape information using aicsimageio package
    czi_aics = AICSImage(filename)
    metadata['Axes_aics'] = czi_aics.dims
    metadata['Shape_aics'] = czi_aics.shape
    metadata['SizeX_aics'] = czi_aics.size_x
    metadata['SizeY_aics'] = czi_aics.size_y
    metadata['SizeC_aics'] = czi_aics.size_c
    metadata['SizeZ_aics'] = czi_aics.size_t
    metadata['SizeT_aics'] = czi_aics.size_t
    metadata['SizeS_aics'] = czi_aics.size_s

    # determine pixel type for CZI array
    metadata['NumPy.dtype'] = czi.dtype

    # check if the CZI image is an RGB image depending on the last dimension entry of axes
    if czi.axes[-1] == 3:
        metadata['isRGB'] = True

    try:
        metadata['PixelType'] = metadatadict_czi['ImageDocument']['Metadata']['Information']['Image']['PixelType']
    except KeyError as e:
        print('Key not found:', e)
        metadata['PixelType'] = None

    metadata['SizeX'] = np.int(metadatadict_czi['ImageDocument']['Metadata']['Information']['Image']['SizeX'])
    metadata['SizeY'] = np.int(metadatadict_czi['ImageDocument']['Metadata']['Information']['Image']['SizeY'])

    try:
        metadata['SizeZ'] = np.int(metadatadict_czi['ImageDocument']['Metadata']['Information']['Image']['SizeZ'])
    except Exception as e:
        #print('Exception:', e)
        if dim2none:
            metadata['SizeZ'] = None
        if not dim2none:
            metadata['SizeZ'] = 1

    try:
        metadata['SizeC'] = np.int(metadatadict_czi['ImageDocument']['Metadata']['Information']['Image']['SizeC'])
    except Exception as e:
        #print('Exception:', e)
        if dim2none:
            metadata['SizeC'] = None
        if not dim2none:
            metadata['SizeC'] = 1

    channels = []
    if metadata['SizeC'] == 1:
        try:
            channels.append(metadatadict_czi['ImageDocument']['Metadata']['DisplaySetting']
                                            ['Channels']['Channel']['ShortName'])
        except Exception as e:
            channels.append(None)

    if metadata['SizeC'] > 1:
        for ch in range(metadata['SizeC']):
            try:
                channels.append(metadatadict_czi['ImageDocument']['Metadata']['DisplaySetting']
                                                ['Channels']['Channel'][ch]['ShortName'])
            except Exception as e:
                print('Exception:', e)
                try:
                    channels.append(metadatadict_czi['ImageDocument']['Metadata']['DisplaySetting']
                                                    ['Channels']['Channel']['ShortName'])
                except Exception as e:
                    print('Exception:', e)
                    channels.append(str(ch))

    metadata['Channels'] = channels

    try:
        metadata['SizeT'] = np.int(metadatadict_czi['ImageDocument']['Metadata']['Information']['Image']['SizeT'])
    except Exception as e:
        #print('Exception:', e)
        if dim2none:
            metadata['SizeT'] = None
        if not dim2none:
            metadata['SizeT'] = 1

    try:
        metadata['SizeM'] = np.int(metadatadict_czi['ImageDocument']['Metadata']['Information']['Image']['SizeM'])
    except Exception as e:
        #print('Exception:', e)
        if dim2none:
            metadatada['SizeM'] = None
        if not dim2none:
            metadata['SizeM'] = 1

    try:
        metadata['SizeB'] = np.int(metadatadict_czi['ImageDocument']['Metadata']['Information']['Image']['SizeB'])
    except Exception as e:
        #print('Exception:', e)
        if dim2none:
            metadatada['SizeB'] = None
        if not dim2none:
            metadata['SizeB'] = 1

    try:
        metadata['SizeS'] = np.int(metadatadict_czi['ImageDocument']['Metadata']['Information']['Image']['SizeS'])
    except Exception as e:
        print('Exception:', e)
        if dim2none:
            metadatada['SizeS'] = None
        if not dim2none:
            metadata['SizeS'] = 1

    try:
        # metadata['Scaling'] = metadatadict_czi['ImageDocument']['Metadata']['Scaling']
        metadata['XScale'] = float(metadatadict_czi['ImageDocument']['Metadata']['Scaling']['Items']['Distance'][0]['Value']) * 1000000
        metadata['YScale'] = float(metadatadict_czi['ImageDocument']['Metadata']['Scaling']['Items']['Distance'][1]['Value']) * 1000000
        metadata['XScale'] = np.round(metadata['XScale'], 3)
        metadata['YScale'] = np.round(metadata['YScale'], 3)
        try:
            metadata['XScaleUnit'] = metadatadict_czi['ImageDocument']['Metadata']['Scaling']['Items']['Distance'][0]['DefaultUnitFormat']
            metadata['YScaleUnit'] = metadatadict_czi['ImageDocument']['Metadata']['Scaling']['Items']['Distance'][1]['DefaultUnitFormat']
        except KeyError as e:
            print('Key not found:', e)
            metadata['XScaleUnit'] = None
            metadata['YScaleUnit'] = None
        try:
            metadata['ZScale'] = float(metadatadict_czi['ImageDocument']['Metadata']['Scaling']['Items']['Distance'][2]['Value']) * 1000000
            metadata['ZScale'] = np.round(metadata['ZScale'], 3)
            try:
                metadata['ZScaleUnit'] = metadatadict_czi['ImageDocument']['Metadata']['Scaling']['Items']['Distance'][2]['DefaultUnitFormat']
            except KeyError as e:
                print('Key not found:', e)
                metadata['ZScaleUnit'] = metadata['XScaleUnit']
        except Exception as e:
            #print('Exception:', e)
            if dim2none:
                metadata['ZScale'] = None
                metadata['ZScaleUnit'] = None
            if not dim2none:
                # set to isotropic scaling if it was single plane only
                metadata['ZScale'] = metadata['XScale']
                metadata['ZScaleUnit'] = metadata['XScaleUnit']
    except Exception as e:
        print('Exception:', e)
        print('Scaling Data could not be found.')

    # try to get software version
    try:
        metadata['SW-Name'] = metadatadict_czi['ImageDocument']['Metadata']['Information']['Application']['Name']
        metadata['SW-Version'] = metadatadict_czi['ImageDocument']['Metadata']['Information']['Application']['Version']
    except KeyError as e:
        print('Key not found:', e)
        metadata['SW-Name'] = None
        metadata['SW-Version'] = None

    try:
        metadata['AcqDate'] = metadatadict_czi['ImageDocument']['Metadata']['Information']['Image']['AcquisitionDateAndTime']
    except KeyError as e:
        print('Key not found:', e)
        metadata['AcqDate'] = None

    # get objective data
    try:
        metadata['ObjName'] = metadatadict_czi['ImageDocument']['Metadata']['Information']['Instrument']['Objectives']['Objective']['Name']
    except KeyError as e:
        print('Key not found:', e)
        metadata['ObjName'] = None

    try:
        metadata['ObjImmersion'] = metadatadict_czi['ImageDocument']['Metadata']['Information']['Instrument']['Objectives']['Objective']['Immersion']
    except KeyError as e:
        print('Key not found:', e)
        metadata['ObjImmersion'] = None

    try:
        metadata['ObjNA'] = np.float(metadatadict_czi['ImageDocument']['Metadata']['Information']
                                     ['Instrument']['Objectives']['Objective']['LensNA'])
    except KeyError as e:
        print('Key not found:', e)
        metadata['ObjNA'] = None

    try:
        metadata['ObjID'] = metadatadict_czi['ImageDocument']['Metadata']['Information']['Instrument']['Objectives']['Objective']['Id']
    except KeyError as e:
        print('Key not found:', e)
        metadata['ObjID'] = None

    try:
        metadata['TubelensMag'] = np.float(metadatadict_czi['ImageDocument']['Metadata']['Information']
                                           ['Instrument']['TubeLenses']['TubeLens']['Magnification'])
    except KeyError as e:
        print('Key not found:', e)
        metadata['TubelensMag'] = None

    try:
        metadata['ObjNominalMag'] = np.float(metadatadict_czi['ImageDocument']['Metadata']['Information']
                                             ['Instrument']['Objectives']['Objective']['NominalMagnification'])
    except KeyError as e:
        metadata['ObjNominalMag'] = None

    try:
        metadata['ObjMag'] = metadata['ObjNominalMag'] * metadata['TubelensMag']
    except KeyError as e:
        print('Key not found:', e)
        metadata['ObjMag'] = None

    # get detector information
    try:
        metadata['DetectorID'] = metadatadict_czi['ImageDocument']['Metadata']['Information']['Instrument']['Detectors']['Detector']['Id']
    except KeyError as e:
        print('Key not found:', e)
        metadata['DetectorID'] = None

    try:
        metadata['DetectorModel'] = metadatadict_czi['ImageDocument']['Metadata']['Information']['Instrument']['Detectors']['Detector']['Name']
    except KeyError as e:
        print('Key not found:', e)
        metadata['DetectorModel'] = None

    try:
        metadata['DetectorName'] = metadatadict_czi['ImageDocument']['Metadata']['Information']['Instrument']['Detectors']['Detector']['Manufacturer']['Model']
    except KeyError as e:
        print('Key not found:', e)
        metadata['DetectorName'] = None

        # delete some key from dict
        # del metadata['Instrument']

    # check for well information
    metadata['Well_ArrayNames'] = []
    metadata['Well_Indices'] = []
    metadata['Well_PositionNames'] = []
    metadata['Well_ColId'] = []
    metadata['Well_RowId'] = []
    metadata['WellCounter'] = None

    try:
        print('Trying to extract Scene and Well information if existing ...')
        # extract well information from the dictionary
        allscenes = metadatadict_czi['ImageDocument']['Metadata']['Information']['Image']['Dimensions']['S']['Scenes']['Scene']

        # loop over all detected scenes
        for s in range(metadata['SizeS']):

            # more than one scene detected
            if metadata['SizeS'] > 1:
                # get the current well and add the array name to the metadata
                well = allscenes[s]
                metadata['Well_ArrayNames'].append(well['ArrayName'])

            # exactly one scene detected (e.g. after split scenes etc.)
            elif metadata['SizeS'] == 1:
                # only get the current well - nor arraynames exist !
                well = allscenes

            # get the well information
            try:
                metadata['Well_Indices'].append(well['Index'])
            except KeyError as e:
                # print('Key not found in Metadata Dictionary:', e)
                metadata['Well_Indices'].append(None)
            try:
                metadata['Well_PositionNames'].append(well['Name'])
            except KeyError as e:
                # print('Key not found in Metadata Dictionary:', e)
                metadata['Well_PositionNames'].append(None)

            # metadata['Well_ColId'].append(well['Shape']['ColumnIndex'])
            # metadata['Well_RowId'].append(well['Shape']['RowIndex'])
            try:
                metadata['Well_ColId'].append(np.int(well['Shape']['ColumnIndex']))
            except KeyError as e:
                print('Key not found in Metadata Dictionary:', e)
                metadata['Well_ColId'].append(None)

            try:
                metadata['Well_RowId'].append(np.int(well['Shape']['RowIndex']))
            except KeyError as e:
                print('Key not found in Metadata Dictionary:', e)
                metadata['Well_RowId'].append(None)

            # more than one scene detected
            if metadata['SizeS'] > 1:
                # count the content of the list, e.g. how many time a certain well was detected
                metadata['WellCounter'] = Counter(metadata['Well_ArrayNames'])

            # exactly one scene detected (e.g. after split scenes etc.)
            elif metadata['SizeS'] == 1:

                # set ArrayNames equal to PositionNames for convenience
                metadata['Well_ArrayNames'] = metadata['Well_PositionNames']

                # count the content of the list, e.g. how many time a certain well was detected
                metadata['WellCounter'] = Counter(metadata['Well_PositionNames'])

            # count the number of different wells
            metadata['NumWells'] = len(metadata['WellCounter'].keys())

    except KeyError as e:
        print('No valid Scene or Well information found:', e)

    # del metadata['Information']
    # del metadata['Scaling']

    # close CZI file
    czi.close()

    # close AICSImage object
    czi_aics.close()

    return metadata